In [77]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [171]:
def tablepd(table):
    rows = table.findAll("tr")
    columns=list(range(len(rows[0])))
    df = pd.DataFrame(columns=columns)
    for i in range(len(rows)):
        tds = rows[i].find_all('td')
        if len(tds) == len(columns):
            values = [ td.text.replace('\n', '').replace('\xa0', ' ') for td in tds ]
            df = df.append(pd.Series(values, index=columns), ignore_index= True)
    
    return df

In [172]:
def yeargames(year,leage):
    url='https://npb.jp/bis/yearly/{}league_{}.html'.format(leage,year)
    columns=['team','game','win','lose','draw','emp','winrate','gamedif']
    
    html = urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    table = soup.findAll("table")[2]
    table2 = table.findAll("table")[0]
    
    df=tablepd(table2)
    df.columns=['team','game','win','lose','draw','emp','winrate','gamedif']
    
    df['year']=year
    df['leage']=leage
    return df

In [186]:
url='https://npb.jp/bis/yearly/'
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")
table = soup.findAll("table")[1]

In [192]:
champsdf=pd.DataFrame(columns=['year','centralchamp','centralteam','pacificchamp','pacificteam'])
k=0
for i in range(8):
    ytb=table.findAll("table")[k]
    ctb=table.findAll("table")[k+1]
    ptb=table.findAll("table")[k+3]
    
    ydf=tablepd(ytb)
    ydf.columns=['year']
    cdf=tablepd(ctb)
    cdf.columns=['centralchamp','centralteam']
    pdf=tablepd(ptb)
    pdf.columns=['pacificchamp','pacificteam']
    df_concat = pd.concat([ydf, cdf],axis=1)
    df_concat = pd.concat([df_concat, pdf],axis=1)
    
    champsdf = pd.concat([champsdf, df_concat],axis=0)
    
    k=k+4

In [193]:
champsdf.loc[champsdf['year'] == '2007',['centralchamp','centralteam']]=['*','中日ドラゴンズ']
champsdf.loc[champsdf['year'] == '2010',['pacificchamp','pacificteam']]=['*','千葉ロッテマリーンズ']
champsdf.loc[champsdf['year'] == '2018',['pacificchamp','pacificteam']]=['*','福岡ソフトバンクホークス']
champsdf.loc[champsdf['year'] == '2019',['pacificchamp','pacificteam']]=['*','福岡ソフトバンクホークス']

In [194]:
champsdf.loc[champsdf['year'] == '2014',['centralchamp','centralteam']]=['','阪神タイガース']
champsdf.loc[champsdf['year'] == '2017',['centralchamp','centralteam']]=['','横浜DeNAベイスターズ']